# Radial Basis Function Network (v 2.0)

## Section 0: Package initialisations and environment configuration

### Import relevant packages:

In [76]:
import numpy as np

# Interactive plotting
import plotly
import plotly.graph_objs as go
import plotly.offline as pyo

### Configure environment:

In [77]:
%config InlineBackend.figure_format = 'retina'
np.set_printoptions(precision=3)

# Activate Plotly Offline for Jupyter
pyo.init_notebook_mode(connected=True)

## Section 1: Loading data

In [87]:
"""
source.npz is a dictionary containing keys 'X' and 'Y', each of which holds an (N x P) matrix.
"""
# Load data
source = np.load("./Data/RBFN/source_ex4_rand.npz")

# Create data and target variables
data = source['Y']
target = source['X']

print 'Shape of data: ', data.shape
print 'Shape of target: ', target.shape

Shape of data:  (979, 2)
Shape of target:  (979, 2)


## Section 2: Defining centres $\mathbf{\mu}$ and variances, $\mathbf{\sigma^2}$:

In [88]:
def calc_causality(data, target, scope=20, show_clusters=False, show_corr_plot=False, show_causality_time_series=False):
    '''
    Calculate the causality index of data -> target.
    Inputs:
        data:               Input values (N x P) (suspected 'effect' variable)
        target:             Output values (N x P)(suspected 'cause' variable)
        scope:              Length of time series subset to perform causality calculations (int)
        show_clusters: If True, scatter plot of clusters will be shown for first set of calculation (Boolean)
    Returns:
        An array of normalised causality index (N - scope)
    '''
    
    def l2(A, axis=None):
        '''
        Calculates the L2-norm of a tensor, at a specified axis.
        Inputs:
            A:    A tensor.
            axis: Summation axis.
        Returns:
            L2-norm of tensor.
        '''
        return np.sqrt(np.sum(np.square(A), axis=1))
    
    def RMS(A):
        '''
        Perform a root-mean square operation.
        Input:
            A: A 1-D array (N)
        Returns:
            Root means square of A.
        '''
        return np.sqrt(np.mean(np.square(A)))
    
    def euclidean_dist(A, B=None):
        '''
        Calculate the euclidean distance for rows in matrix A and rows in matrix B.
        If B is None, calculates distances for rows between matrix A.
        Inputs:
            A: A matrix (a x P)
            B: A matrix (b x P)
        Returns:
            A distance matrix (a x b), indicating the distance of all non-i-th point to the i-th point. 
        ''' 
        # Define input matrices with expanded dimensions
        A_expanded = np.expand_dims(A, 2)

        # Calculate distance of each point and every other point
        if B is None:
            B_expanded = A_expanded
        else:
            B_expanded = np.expand_dims(B, 2)

        return l2(A_expanded - np.transpose(B_expanded, (2, 1, 0)), axis=1)

    def find_cluster_params(data):
        '''
        Given datapoints, find centres and variances for each radial basis.
        Assumptions:
            Each datapoint is itself a radial basis function.
            Variances are assumed to be identical for all basis functions. Calculated as four times the
                average L2 squared distance between all pairwise datapoints.
        Inputs:
            data: Data values (N x P)
        Returns:
            centres:  Centre coordinates for each radial basis (N x P)
            variance: Variance of radial basis (scalar)
        '''
        # Calculate Euclidean distance matrix (N x N)
        distances = euclidean_dist(data)

        # Obtain upper triangular of distances (excluding diagonals)
        distances[np.triu_indices(distances.shape[0], 0)] = 0

        # Calculate variance as four times of [average(Euclidean distance)]^2
        variance = 0.001 *(np.mean(distances) * np.true_divide(np.size(distances), np.sum(distances != 0)))**2

        return data, variance

    def calc_radial_basis_activations(data, centres, variances):
        '''
        Calculates the activations for the hidden radial basis layer.
        Inputs:
            data: Data values (N x P)
            centres: RBF centres (K x P)
            variances: RBF variances (scalar)
        Returns:
            Radial basis activations (N x K)
        '''
        # Calculate Gaussian exponent
        actv = np.exp( - np.divide(euclidean_dist(data, centres)**2, 2 * np.transpose(variances)))
            
        return actv

    def train_RBFN_weights(data, target, centres, variances):
        '''
        Train the weights of a Radial Basis Function Network by solving for values in parameter \mathbf{\alpha}.
        Normalise the activations before solving for \mathbf{\alpha}, 
            and re-normalising the values of \mathbf{\alpha} after.

        Inputs:
            data:   Data values (N x P)
            target: Target values (N x P)
            centres: Cluster centres (K x P)
            variances: Cluster variances (K)
        Returns:
            Trained weights, \mathbf{\alpha} (K x P)
        '''
        # Obtain activations
        activations = calc_radial_basis_activations(data, centres, variances)
        
        # Store the L2-norms of columns of activations in a matrix
        L2 = l2(activations, axis=0)

        # Normalise activation values by their L2-norms
        actvn_norm = np.divide(activations, np.expand_dims(L2, axis=1))
        
        # Solve system of linear equations for alpha
        weights = np.linalg.solve(actvn_norm, target)

        # Return re-normalised alpha
        return np.divide(weights, np.expand_dims(L2, axis=1))

    def RBFN_calc(data, centres, variances, weights):
        '''
        Calculate the output of the trained RBFN.
        Inputs:
            data:      Data values (N x P)
            centres:   RBF centres (K x P)
            variances: RBF variances (scalar)
            weights:   Trained weights (K x P)
        Returns:
            Predicted target value (N x P), assuming data and target have same dimensionality
        '''
        # Calculate radial basis activations
        actv = calc_radial_basis_activations(data, centres, variances)
        
        # Calculate and return predicted output
        return np.matmul(actv, weights)
    
    def visualise_clusters(data, centres, variances):
        '''
        Final result by colouring data points by clusters generated by Mixture of Gaussian algorithm
        Inputs:
            Centres:   Ccoordinates of cluster centres (K x P)
            Variances: Cluster variances (scalar)
        '''
        def calc_ellipse_coordinates(centres, variances):
            '''
            Create x- and y-coordinates for ellipses for each cluster
            Assumptions:
                Dimension of data point is 2
            Returns:
                ellipse: x- and y-coordinates for K ellipses (N x K x D)
            '''
            # Create trace for region to encompass 95% of the points (using Chi-squared critical value)
            # Assuming joint independence and equal marginal variances

            # Chi-squared with df 2 and alpha=5%
            crit_val = 5.991

            # Calculate axes length
            axis_lengths = np.sqrt(variances * crit_val)

            # Calculate coordinates to trace ellipse
            t = np.arange(-np.pi, np.pi + np.pi / 50, np.pi / 50) # Parameter
            x = np.transpose(centres[:,0][:, np.newaxis]) + axis_lengths * np.cos(t)[:, np.newaxis]
            y = np.transpose(centres[:,1][:, np.newaxis]) + axis_lengths * np.sin(t)[:, np.newaxis]

            # Stack x- and y-coordinates along axis=2
            ellipse = np.stack([x, y], axis=2)

            return ellipse

        #######################
        ##  Function begins  ##
        #######################

        # Create ellipse coordinates
        ellipse = calc_ellipse_coordinates(centres, variances)

        # Define colour list as per Plotly's default colour list
        colour_list = np.array(['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b'])

        # Define blank figure
        figure = {
            'data': [],
            'layout': {}
        }
        
        # Create data trace
        data_trace = {
            'x': np.round(data[:, 0], 3),
            'y': np.round(data[:, 1], 3),
            'mode': 'markers',
            'hoverinfo': 'none',
            'marker': {
                'color': colour_list[-1]
            }
        }
        
        figure['data'].append(data_trace)

        for k in range(len(centres)):
            # Create trace for cluster centres
            centre_trace = {
                'x': np.round([centres[k][0]], 3),
                'y': np.round([centres[k][1]], 3),
                'hoverinfo': 'none',
                'mode': 'markers',
                'marker': {
                        'size': 12,
                        'symbol': 'diamond',
                        'color': colour_list[0],
                        'line': {'width': 3}
                    }   
            }

            # Create trace for region encompassing 95% of data points
            variance_trace = {
                'x': ellipse[:,k,:][:,0],
                'y': ellipse[:,k,:][:,1],
                'hoverinfo': 'none',
                'mode': 'lines',
                'marker': {
                    'color': colour_list[0]
                }
            }

            # Add cluster trace
            for trace in [centre_trace, variance_trace]:
                figure['data'].append(trace)

        # Generate figure layout
        figure['layout'] = go.Layout(
            width = 900,
            height = 900,
            showlegend = False,
            title = 'Clusters Visualisation',
            xaxis = {'autorange': True},
            yaxis = {'autorange': True, 'scaleanchor': 'x'}
        )

        return pyo.iplot(figure)
    
    def correlation_plot(target, prediction):
        '''
        Produce a correlation plot between latest time slices of target and prediction.
        Input:
            target:     Target values (N x P)
            prediction: Predicted target values (N x P)
        '''
        # Create trace for points to plot
        point_trace = go.Scatter(
            x = target[:,-1],
            y = prediction[:,-1],
            mode = 'markers',
        )
        
        print target.shape
        print prediction.shape
        
        # Create a line trace for plotting a 45deg line
        line_trace = go.Scatter(
            x = [np.min(target, axis=-1), np.max(target, axis=-1)],
            y = [np.min(target, axis=-1), np.max(target, axis=-1)],
            mode = 'lines',
            line = {
                'dash': 'dash'
            }
        )
        
        # Define layout
        layout = go.Layout(
            width = 800,
            height = 900,
            showlegend = False,
            xaxis = { 'title': 'Target value' },
            yaxis = { 'title': 'Predicted target value', 'scaleanchor': 'x' },
            title = 'Correlation Plot between Target & Predicted Values'
        )
        
        # Define figure
        figure = go.Figure(data=go.Data([point_trace, line_trace]), layout=layout)
        
        # Plot figure offline
        pyo.iplot(figure)
        
        
        
        
    def casuality_time_series(causality):
        '''
        Create a time series plot of the causality indices.
        Inputs:
            causality: Causality index
        '''
        # Define trace
        trace = go.Scatter(
            x = range(len(causality)),
            y = causality,
            mode = 'lines'
        )
        
        # Define layout
        layout = go.Layout(
            title = 'Time Series Plot of Causality Index',
            xaxis = {'title': 'Time Index'},
            yaxis = {'title': 'Causality Index', 'range': [0, 1]},
            showlegend = False,
        )
        
        # Display figure
        pyo.iplot(go.Figure(data=go.Data([trace]), layout=layout))
    
    
    ###################
    # Function Begins #
    ###################
    
    # Error checking
    try:
        assert data.shape[0] >= scope
    except:
        print 'Error: Time series is shorter than scope. Please ensure scope is not shorter than the length of your time series.'
    
    # Define variables
    N = data.shape[0]
    causality = np.zeros(N - scope + 1)
    
    # Visualise clusters to see if centres and variances are appropriate
    if (show_clusters == True):
        centres, variances = find_cluster_params(data=data[:scope]) 
        visualise_clusters(data, centres, variances)

    print 'Calculating causality indices...'
    
    for i in range(N - scope + 1):
        # Define subset of working data and target
        scoped_data = data[i:(scope + i),:]
        scoped_target = target[i:(scope + i),:]
        
        # Define error variable
        error = np.zeros(scope)
        
        for j in range(scope):
            # Obtain centres and variances of RBFs using leave-one-out scoped dataset
            centres, variances = find_cluster_params(data=np.delete(scoped_data, j, 0))
            
            # Train weights of RBFN using leave-one-out scoped dataset
            weights = train_RBFN_weights(
                data = np.delete(scoped_data, j, 0), 
                target = np.delete(scoped_target, j, 0),
                centres = centres,
                variances = variances
            )
            
            # Calculate predicted value of output
            prediction = RBFN_calc(
                data = scoped_data[j,:][np.newaxis, :],
                centres = centres,
                variances = variances,
                weights = weights
            )
            
            # Calculate error
            error[j] = l2(prediction - scoped_target[j,:])
        
        # Calculate \delta
        delta = RMS(error) / RMS(l2(scoped_data - np.mean(scoped_data, axis=0)))
        
        # Calculate causality index
        causality[i] = np.exp( - delta / 5.)
        
    print 'Causality calculations complete!'
    
    # If True, display a correlation plot
    if show_corr_plot == True:
        correlation_plot(target, prediction)
        
    # If True, display a time series of calculated causality
    if show_causality_time_series == True:
        casuality_time_series(causality)
        
    return causality

In [89]:
calc_causality(data=data, target=target, scope=100, show_clusters=True, show_causality_time_series=True);

Calculating causality indices...
Causality calculations complete!


In [90]:
calc_causality(data=target, target=data, scope=100, show_clusters=True, show_causality_time_series=True);

Calculating causality indices...
Causality calculations complete!
